#### I establish the baseline group metrics for adopters and non adopters. In order to determine the impact of the release, I must establish a baseline so that metrics can be compared **relative to** a baseline

In [1]:
import pandas as pd
import os
from pathlib import Path
import duckdb

base_dir = Path('c:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Revenue-Sustainability-Analysis')
data_dir = Path(base_dir / 'Dataset')

# feats incudes all feature_usage history both pre and post release
feats = pd.read_parquet(data_dir / 'feature_usage.parquet')

pd.set_option('display.max_rows',10)

### Partition users into 3 tiered groups: 
* Non-Users (0+ distinct usage days)
* Experimenters (1-2 distinct usage days)
* Adopters (3+ distinct usage days)

First, I want to filter to only those who have any feature_usage history pre release

In [2]:
# Normalize the grain of the table to be one row per sub_id, usage_id
feats = feats.groupby(['subscription_id', 'usage_date', 'feature_name'], as_index=False) \
   .agg({
        'usage_count': 'sum',
        'usage_duration_secs': 'sum',
        'error_count': 'sum',
        'is_beta_feature': 'max'
   })

feats['usage_date'] = pd.to_datetime(feats['usage_date'])

## Validation

# feats[feats.duplicated(subset=['subscription_id', 'usage_date'], keep=False)] \
#             .sort_values(by=['subscription_id', 'usage_date']) \
#             .dropna(subset=['subscription_id', 'usage_date'])

# feats.loc[feats['subscription_id'] == 'S-012ab9'].sort_values(by='usage_date')

### Partition users into 3 tiered groups: 
* Non-Users (0+ distinct usage days)
* Experimenters (1-2 distinct usage days)
* Adopters (3+ distinct usage days)

### Partition users into 3 tiered groups: 
* Non-Users (0+ distinct usage days)
* Experimenters (1-2 distinct usage days)
* Adopters (3+ distinct usage days)

In [3]:
# Pre has all information for both adopters and non adopters 90 days before the release date
# Pre is at the feature usage event level
pre = feats.loc[(feats['usage_date'] < '2023-06-12')].copy()

pre_ids = pre['subscription_id'].dropna().unique()

pre_window = pre.loc[pre['usage_date'] >= '2023-03-14']

### Calculate pre-release metrics at the customer level

Define pre-release window as 90 days prior to June 16, 2023 => March 14, 2023

In [4]:
# Count how many unique days each person used the platform
distinct_usage_days = pre_window.groupby('subscription_id')['usage_date'] \
          .nunique() \
          .reset_index(name='pre_distinct_usage_days')

# Calculate avg gaps between days
# pre_window has the feature_level of the pre release window
pre_window = pre_window.sort_values(['subscription_id', 'usage_date'])
pre_window['gaps'] = pre_window.groupby('subscription_id')['usage_date'] \
    .diff() \
    .dt.days \
    .astype('Int64')

avg_gaps = pre_window.groupby('subscription_id')['gaps'] \
            .mean() \
            .reset_index(name='pre_avg_gaps')

# pre_metrics has the subscription_id level metrics of the pre release window 
pre_metrics = distinct_usage_days.merge(
            avg_gaps,
            on="subscription_id",
            how='outer'
)

# Calculate total usage and error counts for each subscription_id
total_usage = pre_window.groupby('subscription_id')['usage_count'].sum() \
                .reset_index(name='pre_total_usage')

total_usage_duration = pre_window.groupby('subscription_id')['usage_duration_secs'].sum() \
                .reset_index(name='pre_total_usage_duration')

total_error_count = pre_window.groupby('subscription_id')['usage_duration_secs'].sum() \
                .reset_index(name='pre_total_error_count')

# Cust is aggregated at the customer level, still has both adopters and non adopters
pre_metrics = pre_metrics.merge(
        total_usage,
        on='subscription_id',
        how='left'
)

pre_metrics = pre_metrics.merge(
        total_usage_duration,
        on='subscription_id',
        how='left'
)

pre_metrics = pre_metrics.merge(
        total_error_count,
        on='subscription_id',
        how='left'
)

# Calcuate daily averages
pre_metrics['pre_avg_daily_usage'] = (pre_metrics['pre_total_usage'] / pre_metrics['pre_distinct_usage_days']).round(2)
pre_metrics['pre_avg_daily_usage_duration'] = (pre_metrics['pre_total_usage_duration'] / pre_metrics['pre_distinct_usage_days']).round(2)
pre_metrics['pre_avg_daily_error_count'] = (pre_metrics['pre_total_error_count'] / pre_metrics['pre_distinct_usage_days']).round(2)

# For those who are not within the pre release window, fill 0 as inactive behavior 
pre_metrics[['pre_distinct_usage_days', 
             'pre_total_usage', 
             'pre_total_usage_duration',
             'pre_total_error_count',
             'pre_avg_gaps', 
             'pre_avg_daily_usage', 
             'pre_avg_daily_usage_duration', 
             'pre_avg_daily_error_count'
            ]] = pre_metrics[['pre_distinct_usage_days', 
                                'pre_total_usage', 
                                'pre_total_usage_duration',
                                'pre_total_error_count',
                                'pre_avg_gaps', 
                                'pre_avg_daily_usage', 
                                'pre_avg_daily_usage_duration', 
                                'pre_avg_daily_error_count'
            ]].fillna(0)


# Include the subscription_ids who do not have a history within the 90 day pre release window
pre_metrics = pre_metrics.merge(pd.DataFrame({'subscription_id': pre_ids}), on='subscription_id', how='left')

# Create tiers for each customer based on their pre release information
# Partition customers based on distinct_usage_days
pre_metrics['tier'] = pd.cut(
    pre_metrics['pre_distinct_usage_days'],
    bins=[-1,1,3, pre_metrics['pre_distinct_usage_days'].max()],
    labels=['Tier 3 (Low)', 'Tier 2 (Med)', 'Tier 1 (High)']
)

## Validation
# cust.groupby('tier')['distinct_usage_days'].describe()
# cust['tier'].value_counts

print(f"Check for duplicates: {pre_metrics.duplicated(subset='subscription_id').any()}")

Check for duplicates: False


### Partition users into 3 tiered groups: 
* Non-Users (0+ distinct usage days)
* Experimenters (1-2 distinct usage days)
* Adopters (3+ distinct usage days)

### Calculate Post metrics for the short term

In [5]:
# Filters feature history to all feature usage after the release date
post_feat_history = feats.loc[feats['usage_date'] >= '2023-06-12']

# Feature-level table of short term window
short_window = post_feat_history.loc[post_feat_history['usage_date'] < '2023-09-10']

# Filter to only those who interacted with product but only to determine adoption_flag
short_window_newai = short_window.loc[short_window['feature_name'] == 'feature_newai']

# Define adoption_flag for each subscription id. Adoption of feature = 2+ distinct usage dates
# Count how many distinct usage days for each user 
short_usage_ai = short_window_newai.groupby('subscription_id')['usage_date'] \
                                                   .nunique() \
                                                   .reset_index(name='short_distinct_usage_days')

short_usage_ai['adoption_flag'] = (short_usage_ai['short_distinct_usage_days'] >= 2).astype('Int64')

# ~20% of users of the feature are adopters
short_usage_ai['adoption_flag'].mean()

0.19401993355481728

In [6]:
# Calculate Avg gaps post release
short_window = short_window.sort_values(by=['subscription_id','usage_date'])
short_window['gaps'] = short_window.groupby('subscription_id')['usage_date']\
                                             .diff() \
                                             .dt.days \
                                             .astype('Int64')

avg_gaps = short_window.groupby('subscription_id')['gaps'] \
            .mean() \
            .reset_index(name='short_avg_gaps')

In [7]:
# Calculate usage metrics: total_usage, total_usage_duration, avg_daily_usage, avg_daily_usage_duration
total_usage = short_window.groupby('subscription_id')['usage_count'].sum() \
                               .reset_index(name='short_total_usage')

total_usage_duration = short_window.groupby('subscription_id')['usage_duration_secs'].sum() \
                                        .reset_index(name='short_total_usage_duration')

total_error_count = short_window.groupby('subscription_id')['error_count'].sum() \
                                        .reset_index(name='short_total_error_count')

short_metrics = short_usage_ai.merge(avg_gaps, on='subscription_id', how='left')
short_metrics = short_metrics.merge(total_usage, on='subscription_id', how='left') 
short_metrics = short_metrics.merge(total_usage_duration, on='subscription_id', how='left')
short_metrics = short_metrics.merge(total_error_count, on='subscription_id', how='left')

# Calculate averages
short_metrics['short_avg_daily_usage'] = (short_metrics['short_total_usage'] / short_metrics['short_distinct_usage_days']).round(2)
short_metrics['short_avg_daily_usage_duration'] = (short_metrics['short_total_usage_duration'] / short_metrics['short_distinct_usage_days']).round(2)
short_metrics['short_avg_daily_error_count'] = (short_metrics['short_total_error_count'] / short_metrics['short_distinct_usage_days']).round(2)

# Check for dupes, ensure short_metrics is the subscription_id level
print(f"Check for duplicates: {short_metrics.duplicated(subset='subscription_id').any()}")

subs_short = pre_metrics.merge(short_metrics, on='subscription_id', how='left')

Check for duplicates: False


In [8]:
subs_short['tier'].unique()

['Tier 2 (Med)', 'Tier 3 (Low)', 'Tier 1 (High)']
Categories (3, object): ['Tier 3 (Low)' < 'Tier 2 (Med)' < 'Tier 1 (High)']

In [9]:
tier1 = subs_short.loc[subs_short['tier'] == 'Tier 1 (High)']
tier2 = subs_short.loc[subs_short['tier'] == 'Tier 2 (Med)']
tier3 = subs_short.loc[subs_short['tier'] == 'Tier 3 (Low)']

In [10]:
t1_short_metrics = tier1.groupby('adoption_flag').agg({
                        'short_distinct_usage_days': 'mean',
                        'short_avg_gaps': 'mean',
                        'short_total_usage': 'sum',
                        'short_total_usage_duration': 'sum',
                        'short_total_error_count': 'sum',
                        'short_avg_daily_usage': 'mean',
                        'short_avg_daily_usage_duration': 'mean',
                        'short_avg_daily_error_count': 'mean'})

t2_short_metrics = tier2.groupby('adoption_flag').agg({
                        'short_distinct_usage_days': 'mean',
                        'short_avg_gaps': 'mean',
                        'short_total_usage': 'sum',
                        'short_total_usage_duration': 'sum',
                        'short_total_error_count': 'sum',
                        'short_avg_daily_usage': 'mean',
                        'short_avg_daily_usage_duration': 'mean',
                        'short_avg_daily_error_count': 'mean'})

t3_short_metrics = tier3.groupby('adoption_flag').agg({
                        'short_distinct_usage_days': 'mean',
                        'short_avg_gaps': 'mean',
                        'short_total_usage': 'sum',
                        'short_total_usage_duration': 'sum',
                        'short_total_error_count': 'sum',
                        'short_avg_daily_usage': 'mean',
                        'short_avg_daily_usage_duration': 'mean',
                        'short_avg_daily_error_count': 'mean'})

#### Calculate post long term metrics. The window is from 2023-09-10 to present 

In [11]:
# Define long_window as any feature history 90 days after release at the feature level
long_window = post_feat_history.loc[post_feat_history['usage_date'] > '2023-09-10']

In [12]:
# Define adoption_flag for each subscription id. Adoption of feature = 2+ distinct usage dates
# Count how many distinct usage days for each user 
distinct_usage_days = long_window.groupby('subscription_id')['usage_date'] \
                                                   .nunique() \
                                                   .reset_index(name='long_distinct_usage_days')

In [13]:
# Calculate Avg gaps post release
long_window = long_window.sort_values(by=['subscription_id','usage_date'])
long_window['gaps'] = long_window.groupby('subscription_id')['usage_date']\
                                             .diff() \
                                             .dt.days \
                                             .astype('Int64')

avg_gaps = long_window.groupby('subscription_id')['gaps'] \
            .mean() \
            .reset_index(name='long_avg_gaps')

In [14]:
# Calculate usage metrics: total_usage, total_usage_duration, avg_daily_usage, avg_daily_usage_duration
total_usage = long_window.groupby('subscription_id')['usage_count'].sum() \
                               .reset_index(name='long_total_usage')

total_usage_duration = long_window.groupby('subscription_id')['usage_duration_secs'].sum() \
                                        .reset_index(name='long_total_usage_duration')

total_error_count = long_window.groupby('subscription_id')['error_count'].sum() \
                                        .reset_index(name='long_total_error_count')

long_metrics = distinct_usage_days.merge(short_usage_ai, on='subscription_id', how='right') \
                                   .drop(columns='short_distinct_usage_days')

long_metrics = long_metrics.merge(avg_gaps, on='subscription_id', how='left')
long_metrics = long_metrics.merge(total_usage, on='subscription_id', how='left') 
long_metrics = long_metrics.merge(total_usage_duration, on='subscription_id', how='left')
long_metrics = long_metrics.merge(total_error_count, on='subscription_id', how='left')

# Calculate averages
long_metrics['long_avg_daily_usage'] = (long_metrics['long_total_usage'] / long_metrics['long_distinct_usage_days']).round(2)
long_metrics['long_avg_daily_usage_duration'] = (long_metrics['long_total_usage_duration'] / long_metrics['long_distinct_usage_days']).round(2)
long_metrics['long_avg_daily_error_count'] = (long_metrics['long_total_error_count'] / long_metrics['long_distinct_usage_days']).round(2)

# Check for dupes, ensure long_metrics is the subscription_id level
print(f"Check for duplicates: {long_metrics.duplicated(subset='subscription_id').any()}")

subs_long = pre_metrics.merge(long_metrics, on='subscription_id', how='left')

Check for duplicates: False


In [15]:
tier1 = subs_long.loc[subs_long['tier'] == 'Tier 1 (High)']
tier2 = subs_long.loc[subs_long['tier'] == 'Tier 2 (Med)']
tier3 = subs_long.loc[subs_long['tier'] == 'Tier 3 (Low)']

t1_long_metrics = tier1.groupby('adoption_flag').agg({
                        'long_distinct_usage_days': 'mean',
                        'long_avg_gaps': 'mean',
                        'long_total_usage': 'sum',
                        'long_total_usage_duration': 'sum',
                        'long_total_error_count': 'sum',
                        'long_avg_daily_usage': 'mean',
                        'long_avg_daily_usage_duration': 'mean',
                        'long_avg_daily_error_count': 'mean'})

t2_long_metrics = tier2.groupby('adoption_flag').agg({
                        'long_distinct_usage_days': 'mean',
                        'long_avg_gaps': 'mean',
                        'long_total_usage': 'sum',
                        'long_total_usage_duration': 'sum',
                        'long_total_error_count': 'sum',
                        'long_avg_daily_usage': 'mean',
                        'long_avg_daily_usage_duration': 'mean',
                        'long_avg_daily_error_count': 'mean'})

t2_long_metrics = tier3.groupby('adoption_flag').agg({
                        'long_distinct_usage_days': 'mean',
                        'long_avg_gaps': 'mean',
                        'long_total_usage': 'sum',
                        'long_total_usage_duration': 'sum',
                        'long_total_error_count': 'sum',
                        'long_avg_daily_usage': 'mean',
                        'long_avg_daily_usage_duration': 'mean',
                        'long_avg_daily_error_count': 'mean'})

### Calculate deltas to compare post-short and post-long to pre-release

#### Short Deltas

In [16]:
short_deltas = pd.DataFrame(subs_short[['subscription_id', 'tier', 'adoption_flag']])

short_deltas['distinct_usage_days'] = subs_short['short_distinct_usage_days'] - subs_short['pre_distinct_usage_days']
short_deltas['avg_gaps'] = subs_short['short_avg_gaps'] - subs_short['pre_avg_gaps']
short_deltas['total_usage'] = subs_short['short_total_usage'] - subs_short['pre_total_usage']
short_deltas['total_usage_duration'] = subs_short['short_total_usage_duration'] - subs_short['pre_total_usage_duration']
short_deltas['total_error_count'] = subs_short['short_total_error_count'] - subs_short['pre_total_error_count']
short_deltas['avg_daily_usage'] = subs_short['short_avg_daily_usage'] - subs_short['pre_avg_daily_usage']
short_deltas['avg_daily_usage_duration'] = subs_short['short_avg_daily_usage_duration'] - subs_short['pre_avg_daily_usage_duration']
short_deltas['avg_daily_error_count'] = subs_short['short_avg_daily_error_count'] - subs_short['pre_avg_daily_error_count']

short_deltas = short_deltas.groupby(['tier','adoption_flag']).agg({
                        'distinct_usage_days': 'mean',
                        'avg_gaps': 'mean',
                        'total_usage': 'sum',
                        'total_usage_duration': 'sum',
                        'total_error_count': 'sum',
                        'avg_daily_usage': 'mean',
                        'avg_daily_usage_duration': 'mean',
                        'avg_daily_error_count': 'mean'}).round(2)

C:\Users\henry\AppData\Local\Temp\ipykernel_27396\187180989.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  short_deltas = short_deltas.groupby(['tier','adoption_flag']).agg({


In [17]:
short_deltas

distinct_usage_days  avg_gaps  total_usage  \
tier          adoption_flag                                               
Tier 3 (Low)  0                             0.00      28.0         2949   
              1                             1.15      21.3         2086   
Tier 2 (Med)  0                            -1.19     -5.08         -461   
              1                             0.03     -3.77          584   
Tier 1 (High) 0                            -3.11       3.5         -124   
              1                            -2.00       7.4          -10   

                             total_usage_duration  total_error_count  \
tier          adoption_flag                                            
Tier 3 (Low)  0                            876478           -1184287   
              1                            632752            -240285   
Tier 2 (Med)  0                           -157795           -1373065   
              1                            195068            -464851   
Tier 1 (High) 0                            -37245             -91917   
              1                             -5139             -42449   

                             avg_daily_usage  avg_daily_usage_duration  \
tier          adoption_flag                                              
Tier 3 (Low)  0                         7.12                    2117.1   
              1                         4.83                   1639.15   
Tier 2 (Med)  0                         8.88                   2544.79   
              1                         3.53                   1140.75   
Tier 1 (High) 0                        10.48                   3593.53   
              1                         7.15                    1608.8   

                             avg_daily_error_count  
tier          adoption_flag                         
Tier 3 (Low)  0                            -2860.6  
              1                           -2356.58  
Tier 2 (Med)  0                           -2693.44  
              1                           -2842.05  
Tier 1 (High) 0                           -2481.13  
              1                            -2122.2

Tier 3 (Low)
Frequency/Presence
* Usage Days: adopters have slightly more days
* avg_gaps: adopters have less gaps between engagement

Engagement Depth/Intensity
* total_usage: non adopters have more total usage by ~50%. And also more total usage duration
* total_error_count: non adopters have more **reduction** in total error count
* avg_daily_usage: non adopters hvae more avg daily usage, usage duration, less in avg daily error count

Although adoption does lead to shallow reactivation (distinct usage days increase, avg_gaps decrease), but does not translate into sustained or deep customer engagement in the platform since average engagement decreased.
Usage intensity remains low among adopters, indicating that the feature may return brief reactivation.

------------------------------------------------------

Tier 2 (Med)
* Distinct usage days: Decreases for non adopters but no change for adopters. 
* Avg gaps: Improves for both groups, non adopters improve more.
* Usage decreases from a volume for baseline. Adoption yields more total usage

Non Adopters: Usage total is gradually decreasing on the platform; however, average gaps is also decreasing, which may signify potential survivorship bias. However, intensity of engagement increases, which may signify futher survivorship or power users engagement.

Adopters: Usage total increases but their engagement 
    
------------------------------------------------------

Tier 1 (High)
* Distinct Usage Days: 
* Adopters have more avg gaps
* Non adopters lost usage from a volume perspective but had a greater reduction in total error count
* Non adopters have higher daily avg usage and error count

Users show contraction rather than growth post-release since total usage and activity days decline for both adopters and non adopters. While non adopters show higher average daily usage,
this can be driven by concentration among fewer remaining sessions rather than expanded engagement on the platform, which suggests intensity without scale. 




### Long Deltas


In [18]:
long_deltas = pd.DataFrame(subs_long[['subscription_id', 'tier', 'adoption_flag']])

long_deltas['distinct_usage_days'] = subs_long['long_distinct_usage_days'] - subs_long['pre_distinct_usage_days']
long_deltas['avg_gaps'] = subs_long['long_avg_gaps'] - subs_long['pre_avg_gaps']
long_deltas['total_usage'] = subs_long['long_total_usage'] - subs_long['pre_total_usage']
long_deltas['total_usage_duration'] = subs_long['long_total_usage_duration'] - subs_long['pre_total_usage_duration']
long_deltas['total_error_count'] = subs_long['long_total_error_count'] - subs_long['pre_total_error_count']
long_deltas['avg_daily_usage'] = subs_long['long_avg_daily_usage'] - subs_long['pre_avg_daily_usage']
long_deltas['avg_daily_usage_duration'] = subs_long['long_avg_daily_usage_duration'] - subs_long['pre_avg_daily_usage_duration']
long_deltas['avg_daily_error_count'] = subs_long['long_avg_daily_error_count'] - subs_long['pre_avg_daily_error_count']

long_deltas = long_deltas.groupby(['tier','adoption_flag']).agg({
                        'distinct_usage_days': 'mean',
                        'avg_gaps': 'mean',
                        'total_usage': 'sum',
                        'total_usage_duration': 'sum',
                        'total_error_count': 'sum',
                        'avg_daily_usage': 'mean',
                        'avg_daily_usage_duration': 'mean',
                        'avg_daily_error_count': 'mean'}).round(2)

C:\Users\henry\AppData\Local\Temp\ipykernel_27396\4061123466.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  long_deltas = long_deltas.groupby(['tier','adoption_flag']).agg({


In [19]:
long_deltas

distinct_usage_days  avg_gaps  total_usage  \
tier          adoption_flag                                               
Tier 3 (Low)  0                             4.61     81.38        16984   
              1                             4.81     76.38         4516   
Tier 2 (Med)  0                             4.19     40.03         8745   
              1                             4.46     39.84         2978   
Tier 1 (High) 0                             2.22     66.03          239   
              1                             4.00     21.16          203   

                             total_usage_duration  total_error_count  \
tier          adoption_flag                                            
Tier 3 (Low)  0                           5175188           -1181289   
              1                           1500203            -240119   
Tier 2 (Med)  0                           2591442           -1368141   
              1                            865281            -445673   
Tier 1 (High) 0                             58875             -91893   
              1                             87565             -33697   

                             avg_daily_usage  avg_daily_usage_duration  \
tier          adoption_flag                                              
Tier 3 (Low)  0                        -0.21                   -106.57   
              1                         0.31                    564.63   
Tier 2 (Med)  0                        -0.19                     -3.02   
              1                        -0.19                    -13.52   
Tier 1 (High) 0                         0.94                     45.56   
              1                         2.78                   1720.68   

                             avg_daily_error_count  
tier          adoption_flag                         
Tier 3 (Low)  0                           -2876.51  
              1                           -2356.77  
Tier 2 (Med)  0                           -2696.08  
              1                           -2828.42  
Tier 1 (High) 0                           -2481.12  
              1                           -2106.47

Tier 3: Users show surface-level improvement without scale, since adopters intensity increases (avg daily usage) but total usage and average gaps remain low compared to non adopters
this suggests, episodes of engagement, but non-habit forming adoption.

Tier 2: This suggests broad contraction regardless of adoption. Both adopters and non adopters lose average daily usage and only experience minimal imporvements in total usage. This suggests the release may have failed to create durable behavior change for mid-engagement users.

Tier 1: Adopters of the feature have much larger reduced gaps and continue to have higher daily average usage compared to non adopters as well as higher total usage and less average gaps.
Adopters of the feature for high-engagement users seem to have deepened engagement rather than sustaining baseline behavior.